In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
     

from peft import PeftModel, PeftConfig

In [2]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()
     

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

In [4]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))
     

In [5]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: -->첫째, 물을 자주 마셔서 수분을 섭취하십시오. 둘째, 과일과 야채를 충분하게 섭취하십시오.셋째, 일년에 한번정도 종합 건강검진을 받는 것이 좋습니다. Q


# 1. 나만의 데이터셋 만들기

In [6]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

In [19]:
dataPath = "./dataset/"

In [15]:
datasetName = "data.CSV"
jsonFileName = "data.jsonl"

In [25]:
indataset = []

In [24]:
def csv_to_json(csv_file_path, json_file_path):
    df = pd.read_csv(csv_file_path,encoding='cp949')

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'instruction': row['instruction'], 'output': row['output']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)


In [33]:
import jsonlines
from datasets import Dataset

# JSON Lines 파일 경로 설정
json_file_path = './dataset/data.jsonl'
dataPath = './dataset/'

# 데이터를 로드하고 저장할 두 리스트 초기화
instructions = []
outputs = []

# JSON Lines 파일을 읽고 각 데이터 포인트를 리스트에 저장
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
        instructions.append(line["instruction"])
        outputs.append(line["output"])

# 데이터셋 생성 및 저장
dataset = Dataset.from_dict({
    "instruction": instructions,
    "output": outputs
})
dataset.save_to_disk(dataPath)


# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 리스트를 이용해 Dataset 객체 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)


데이터셋 확인
['<s>[INST] 노하림은 몇살이야? [/INST] 노하림은 23살입니다. </s>', '<s>[INST] 노하림은 어디살아? [/INST] 노하림은 서울 창동에 살아요. </s>', '<s>[INST] 노하림은 무슨 학교다녀? [/INST] 노하림은 덕성여대 컴퓨터공학과야. </s>', '<s>[INST] 남서현은 몇살이야? [/INST] 남서현은 24살이다. </s>', '<s>[INST] 남서현은 어디살아? [/INST] 남서현은 강원도 원주 살고 지금은 기숙사 살아 </s>']


Saving the dataset (0/1 shards):   0%|          | 0/6 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 6
})


In [34]:
indataset.push_to_hub("ho1iday/harimKo")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ho1iday/harimKo/commit/8c6ee6a12d0a11a12b7032b8f7a836ad75439800', commit_message='Upload dataset', commit_description='', oid='8c6ee6a12d0a11a12b7032b8f7a836ad75439800', pr_url=None, pr_revision=None, pr_num=None)

# 2. 데이터셋 로드 하기

In [35]:
from datasets import load_dataset

data = load_dataset("ho1iday/harimKo")

Generating train split:   0%|          | 0/6 [00:00<?, ? examples/s]

In [36]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6
    })
})

In [38]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

KeyError: 'instruction'

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [ ]:
data['train'][0]['text']

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# 3. 학습하기